# TPP Backend API Integration

This notebook demonstrates how to interact with the TPP monitoring backend API from Python/Jupyter.

**Features:**
- Health check and server status
- Upload monitoring data programmatically
- Fetch uploaded data
- Data validation and testing

## Prerequisites

Make sure the backend server is running:
```bash
cd backend
npm start
```

Server should be running at: http://localhost:3000

## 1. Setup & Import Libraries

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime

# API configuration
BASE_URL = "http://localhost:3000"
HEALTH_ENDPOINT = f"{BASE_URL}/health"
UPLOAD_ENDPOINT = f"{BASE_URL}/upload"

print(f"✓ API Base URL: {BASE_URL}")

## 2. Health Check

Verify the backend server is running and responsive.

In [ ]:
try:
    response = requests.get(HEALTH_ENDPOINT, timeout=5)
    if response.status_code == 200:
        data = response.json()
        print("✅ Server is healthy!")
        print(f"   Status: {data.get('status')}")
        print(f"   Message: {data.get('message')}")
        print(f"   Timestamp: {data.get('timestamp')}")
    else:
        print(f"⚠️  Server returned status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to server. Make sure it's running at", BASE_URL)
except Exception as e:
    print(f"❌ Error: {e}")

## 3. Upload Data from Python

Upload heart rate data programmatically using the API.

In [ ]:
# Load sample data
with open('../sample-data/heart-rate-sample.json', 'r') as f:
    sample_data = json.load(f)

# Upload via API
files = {
    'file': ('heart-rate-sample.json', json.dumps(sample_data), 'application/json')
}

try:
    response = requests.post(UPLOAD_ENDPOINT, files=files, timeout=10)
    if response.status_code == 200:
        result = response.json()
        print("✅ Upload successful!")
        print(f"   Records uploaded: {result.get('count', 'N/A')}")
        print(f"   Message: {result.get('message', 'N/A')}")
    else:
        print(f"❌ Upload failed with status: {response.status_code}")
        print(f"   Response: {response.text}")
except Exception as e:
    print(f"❌ Error during upload: {e}")

## 4. Create & Upload Synthetic Data

Generate synthetic heart rate data for testing.

In [ ]:
import numpy as np

# Generate synthetic data
np.random.seed(42)
num_records = 20

synthetic_data = []
base_time = datetime.now()

for i in range(num_records):
    record = {
        "timestamp": (base_time.replace(hour=8+i//2, minute=(i%2)*30)).isoformat(),
        "heartRate": int(np.random.normal(75, 10)),
        "hrv": int(np.random.normal(50, 15)),
        "activity": np.random.choice(['resting', 'walking', 'exercise']),
        "device": "Synthetic Generator"
    }
    synthetic_data.append(record)

# Display sample
print(f"Generated {len(synthetic_data)} synthetic records\n")
df_synthetic = pd.DataFrame(synthetic_data)
print(df_synthetic.head())

# Upload synthetic data
files = {
    'file': ('synthetic-data.json', json.dumps(synthetic_data), 'application/json')
}

try:
    response = requests.post(UPLOAD_ENDPOINT, files=files, timeout=10)
    if response.status_code == 200:
        print("\n✅ Synthetic data uploaded successfully!")
    else:
        print(f"\n❌ Upload failed: {response.status_code}")
except Exception as e:
    print(f"\n❌ Error: {e}")

## 5. Helper Functions

Utility functions for common API operations.

In [ ]:
def check_server_health():
    """Check if the backend server is running and healthy."""
    try:
        response = requests.get(HEALTH_ENDPOINT, timeout=5)
        return response.status_code == 200
    except:
        return False

def upload_json_data(data, filename="data.json"):
    """Upload JSON data to the backend API."""
    files = {
        'file': (filename, json.dumps(data), 'application/json')
    }
    try:
        response = requests.post(UPLOAD_ENDPOINT, files=files, timeout=10)
        return response.status_code == 200, response.json() if response.status_code == 200 else None
    except Exception as e:
        return False, str(e)

def upload_csv_data(df, filename="data.csv"):
    """Upload DataFrame as CSV to the backend API."""
    csv_content = df.to_csv(index=False)
    files = {
        'file': (filename, csv_content, 'text/csv')
    }
    try:
        response = requests.post(UPLOAD_ENDPOINT, files=files, timeout=10)
        return response.status_code == 200, response.json() if response.status_code == 200 else None
    except Exception as e:
        return False, str(e)

print("✓ Helper functions defined")
print("  - check_server_health()")
print("  - upload_json_data(data, filename)")
print("  - upload_csv_data(df, filename)")

## 6. Test Upload with DataFrame

Upload a pandas DataFrame directly to the API.

In [ ]:
# Test with helper function
if check_server_health():
    print("✅ Server is healthy, proceeding with upload...\n")
    
    # Upload the synthetic DataFrame
    success, result = upload_csv_data(df_synthetic, "test-upload.csv")
    
    if success:
        print("✅ CSV upload successful!")
        print(f"   Result: {result}")
    else:
        print(f"❌ CSV upload failed: {result}")
else:
    print("❌ Server is not running. Start it with: npm start")